In [1]:
!git clone https://github.com/hodakamori/torch-tutorial

Cloning into 'torch-tutorial'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 64 (delta 16), reused 58 (delta 13), pack-reused 0
Receiving objects: 100% (64/64), 9.53 MiB | 28.88 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [3]:
!pip install graph-transformer-pytorch rdkit MDAnalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 908.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 kB 28.1 MB/s eta 0:00:00


In [4]:
!cp torch-tutorial/diffusion_two_for_one/* .

In [5]:
import torch
import random
from torch.utils.data import DataLoader
from dataset import CGCoordsDataset
from model import Net
from utils import add_diffusion_noise

In [6]:
device = torch.device("cuda")
topology_path = "./ala2_cg.pdb"
traj_path = "./ala2_cg.xtc"
dataset = CGCoordsDataset(topology_path, traj_path)
print(len(dataset))

MAX_EPOCHS = 5
BATCH_SIZE = 1024
MAX_NOISE_LEVEL = 25

1000


/usr/local/lib/python3.10/dist-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/usr/local/lib/python3.10/dist-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: X
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [7]:
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
bonds = dataset.bonds
model = Net(num_atoms=5, num_node_features=64)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=MAX_EPOCHS, eta_min=1e-5
)
loss_func = torch.nn.MSELoss()

for epoch in range(MAX_EPOCHS):
    history = []
    for iter, (indices, coords) in enumerate(dataloader):
        coords = coords.to(device)
        coords.requires_grad_()
        noise_levels = torch.randint(
            1, MAX_NOISE_LEVEL + 1, (coords.shape[0], coords.shape[1], 1)
        )
        noised_coordinates = add_diffusion_noise(coords, noise_levels=noise_levels)
        noise_true = noised_coordinates - coords
        energy = model(indices, noised_coordinates, bonds, noise_levels=noise_levels)
        if coords.grad is not None:
            coords.grad.zero_()
        energy.backward(retain_graph=True)
        noise_pred = coords.grad
        loss = loss_func(noise_true, noise_pred).mean()
        print(f"{epoch=}, {iter=}, loss={loss.detach().cpu().numpy()}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        history.append(loss.detach().cpu().numpy())